In [ ]:
!pip install langchain
!pip install jq
!pip install sentence-transformers
!pip install chromadb

In [1]:
from langchain_community.document_loaders import JSONLoader

In [2]:
diff_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

msg_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].msg',
    text_content=False)

msg_data = msg_loader.load()

language_loader = JSONLoader(
    file_path='../data/rag_data.json',
    jq_schema='.[].language',
    text_content=False)

language_data = language_loader.load()

In [3]:
# 1-1000 data use java splitter, 1000-2000 use cpp splitter, 2000-3000 use csharp splitter, 3000-4000 use python splitter, 4000-5000 use javascript splitter
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter

languages = [Language.JAVA, Language.CPP, Language.CSHARP, Language.PYTHON, Language.JS]
splitters = [RecursiveCharacterTextSplitter.from_language(language, chunk_size=300, chunk_overlap=0) for language in languages]

language_dict = {'java': 0, 'cpp': 1, 'csharp': 2, 'python': 3, 'javascript': 4}

In [4]:
from tqdm import tqdm
diff_split = []
for i, doc in tqdm(enumerate(diff_data), total=len(diff_data), desc="Processing documents"):
    diff_split += splitters[language_dict[language_data[i].page_content]].split_documents([doc])

In [5]:
len(diff_split)

In [10]:
diff_split[0]

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [7]:
from langchain_community.vectorstores import Chroma

In [10]:
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [9]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(diff_data, embeddings, persist_directory="./chroma_db")

In [12]:
db.persist()

In [11]:
test_diff_loader = JSONLoader(
    file_path='../data/msg_nngen_nmt_codebert_chatgpt.json',
    jq_schema='.[].diff',
    text_content=False)

test_diff_data = test_diff_loader.load()

In [12]:
def similarity_search(documents):
    # Initialize an empty dictionary to store aggregate scores for each candidate ID
    aggregate_scores = {}
    
    # Iterate through each document in the documents list
    for document in documents:
        # Apply similarity search function to the document
        results = db.similarity_search_with_relevance_scores(document.page_content, score_threshold=0.0)
        # Iterate through the results for each document
        for candidate_doc in results:
            id = candidate_doc[0].metadata['seq_num']
            score = candidate_doc[1]
            # Update the aggregate score for the candidate ID
            aggregate_scores[id] = aggregate_scores.get(id, 0) + score
    
    # Find the candidate ID with the highest aggregate score
    max_candidate_id = max(aggregate_scores, key=aggregate_scores.get)
    return max_candidate_id - 1

In [13]:
retriever = db.as_retriever()

In [16]:
test_diff_data[0].page_content

In [19]:
retriever.get_relevant_documents(test_diff_data[0].page_content)[0].metadata['seq_num']

In [22]:
from tqdm import tqdm

In [23]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data[:100]), total=len(test_diff_data[:100]), desc="Processing documents"):
    query = test_data.page_content
    similar_diff.append(retriever.get_relevant_documents(query)[0])

In [24]:
len(similar_diff)

In [25]:
msg_data[similar_diff[0].metadata['seq_num']-1].page_content

In [26]:
import json
data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data[:100]):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']-1].page_content,
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

In [27]:
# Write the data to a JSON file
with open('../data/test_data_with_rag_100.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [50]:
query =test_diff_data[0].page_content
documents = splitters[0].create_documents([query])

In [51]:
docs = retriever.get_relevant_documents(documents[0].page_content)

In [57]:
db.similarity_search_with_relevance_scores(documents[0].page_content)[1]

In [87]:
with open('../data/msg_nngen_nmt_codebert_chatgpt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/msg_nngen_nmt_codebert_chatgpt_rag.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [88]:
len(gpt_msg)